## 4. Clustering using K-Means
To group stations based on their extracted features, we apply k-means clustering to the feature space spanned by DPI, WSD, and SDI. Prior to clustering, all features are standardised to ensure equal weighting.

The number of clusters is fixed to $k=3$:

In [ ]:
N_CLUSTERS = 3

reflecting the conceptual distinction between **recreational-oriented**, **utilitarian-oriented**, and **mixed** usage types introduced earlier. Rather than optimising purely geometric separation metrics, this choice prioritises **interpretability** and alignment with the underlying traffic behaviour assumptions.

In [ ]:
from data_io.loader.data_loader import DataLoader
from analysis.visualization.characterisation.clustering import kmeans_clustering
from analysis.visualization.characterisation.features import build_feature_df


dl = DataLoader(city="Stadt_Heidelberg")

X = build_feature_df(dl)

clustering = kmeans_clustering(features=X, k=N_CLUSTERS)

clustering.drop(["valid"])

In [ ]:
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

X_valid = X.filter(pl.col("valid") == True)
X_feat = X_valid.drop(["station", "valid"]).to_numpy()
X_scaled = StandardScaler().fit_transform(X_feat)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init=20)
labels = kmeans.fit_predict(X_scaled)

X_valid = X_valid.with_columns(pl.Series("cluster", labels))

X = X.join(
    X_valid.select(["station", "cluster"]),
    on="station",
    how="left"
)

X.drop(["station", "valid"]).group_by("cluster").mean()

### Feature discrimination across clusters

Before we continue let's check how much the features to discriminate between the clusters. To assess this we inspect their distributions using boxplots.

In [ ]:
from analysis.visualization.characterisation.plotting import plot_feature_boxplots

EXCLUDE = {"station", "valid", "cluster", "date"}

FEATURES = [
    c for c in X.columns
    if c not in EXCLUDE and X[c].dtype in (pl.Float32, pl.Float64)
]

plot_feature_boxplots(
    X,
    features=FEATURES,
    cluster_col="cluster",
    clusters=(0, 1, 2),
)

**DPI:** shows clear separation between clusters, with one cluster exhibiting pronounced weekday double-peak structures typical of commuting traffic, while another cluster displays consistently low DPI values, indicating the absence of work-related patterns.

**WSD:** strongly differentiates the clusters, separating stations with substantial weekday–weekend contrasts from those with similar daily profiles throughout the week.

**SDI:** separation is less pronounced, it supports the distinction between clusters with stable year-round usage and those with stronger seasonal dependence.

### Feature correlation

In [ ]:
X.select(FEATURES).corr()

We observe a strong correlation between **DPI** and **WSD**.  
High **DPI** ⇒ pronounced weekday commuting peaks ⇒ strong structural differences between weekday and weekend profiles (absence of commuting peaks on weekends) ⇒ high **WSD**.

Although correlated, the features are not redundant: **DPI** captures the strength of commuting-related peak structure, while **WSD** quantifies the overall weekday–weekend shape difference. 

### PCA
Reducing the standardized feature space to two dimensions using PCA. This allows us to further explore the feature space 
Using the first two components the majority of variance is explained. 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Explained variance:", pca.explained_variance_ratio_)

X_pca_pl = (
    X
    .with_columns([
        pl.Series("PC1", X_pca[:, 0]),
        pl.Series("PC2", X_pca[:, 1]),
    ])
)
X_pca_pl

In [ ]:
from analysis.visualization.characterisation.plotting import plot_pca_clusters

plot_pca_clusters(
    X_pca_pl,
    cluster_col="cluster",
    annotate=True
)

The projection reveals a clear separation of stations primarily along the first principal component (PC1). This axis reflects the contrast between stations with pronounced weekday commuting structure and those with more homogeneous, recreational traffic patterns. 

The second principal component (PC2) captures secondary variations, including differences in seasonal sensitivity and intermediate usage characteristics.